## PyTorch Training

Uses the Trainer included in Hugging Face `transformers` (backed by `accelerate`) since it mitigates a lot of annoying boilerplate.


In [1]:
import os
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from transformers import Trainer, TrainingArguments
from transformers.trainer_callback import TrainerCallback

In [2]:
df = (
    pl.scan_parquet(
        "movie_data_plus_embeds_all.parquet"
    )
    .select(["tconst", "averageRating", "embedding"])
    .with_columns(averageRating=pl.col("averageRating").cast(pl.Float32))
    .collect()
    .sample(fraction=1.0, shuffle=True, seed=42)
)

df

tconst,averageRating,embedding
str,f32,"array[f32, 768]"
"""tt0173052""",4.1,"[0.046187, 0.006053, … 0.011911]"
"""tt0266288""",7.4,"[-0.004875, -0.046969, … 0.017516]"
"""tt6263490""",4.3,"[0.005363, -0.018672, … 0.015112]"
"""tt10049110""",7.8,"[-0.009997, -0.029303, … 0.037793]"
"""tt5761612""",3.8,"[0.020259, -0.031869, … -0.01841]"
…,…,…
"""tt0079376""",6.2,"[0.062672, -0.009446, … 0.019441]"
"""tt1161064""",3.2,"[0.022779, 0.053063, … -0.009691]"
"""tt0179526""",5.7,"[0.001937, 0.003111, … -0.002453]"


In [3]:
device = "cpu"
n_test = 20000

X_train = torch.from_numpy(df[:-n_test]["embedding"].to_numpy().copy()).to(device)
X_test = torch.from_numpy(df[-n_test:]["embedding"].to_numpy().copy()).to(device)

y_train = torch.from_numpy(df[:-n_test]["averageRating"].to_numpy().copy()).to(device)
y_test = torch.from_numpy(df[-n_test:]["averageRating"].to_numpy().copy()).to(device)

y_train

tensor([4.1000, 7.4000, 4.3000,  ..., 6.4000, 6.0000, 6.5000])

In [4]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [5]:
class RatingsModel(nn.Module):
    def __init__(self, linear_dims=256, num_layers=6):
        super().__init__()
        
        dims = [768] + [linear_dims] * num_layers
        self.mlp = nn.ModuleList([
            nn.Sequential(
                nn.Linear(dims[i], dims[i+1]),
                nn.GELU(),
                nn.BatchNorm1d(dims[i+1]),
                nn.Dropout(0.6)
            ) for i in range(len(dims)-1)
        ])
        
        self.output = nn.Linear(dims[-1], 1)

    def forward(self, x, targets=None):
        for layer in self.mlp:
            x = layer(x)

        return self.output(x).squeeze()  # return 1D output

In [6]:
model = RatingsModel()
_ = model.to(device)
model

RatingsModel(
  (mlp): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=768, out_features=256, bias=True)
      (1): GELU(approximate='none')
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.6, inplace=False)
    )
    (1-5): 5 x Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): GELU(approximate='none')
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.6, inplace=False)
    )
  )
  (output): Linear(in_features=256, out_features=1, bias=True)
)

In [7]:
total_params = sum(p.numel() for p in model.parameters())
total_params

529153

Validation loss doesn't play nice with the `Trainer` out of the boss, so need [some tweaks](https://discuss.huggingface.co/t/no-log-for-validation-loss-during-training-with-trainer/40094/3).


In [8]:
def collate_fn(examples):
    inputs = torch.stack([f[0] for f in examples])
    targets = torch.stack([f[1] for f in examples])

    return {"x": inputs, "targets": targets}


class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=0):
        outputs = model(**inputs)
        # loss = nn.L1Loss()(outputs, inputs["targets"])  # L1 loss is MAE
        loss = nn.MSELoss()(outputs, inputs["targets"])

        return (loss, outputs) if return_outputs else loss


In [9]:
training_args = TrainingArguments(
    learning_rate=2e-3,
    lr_scheduler_type="cosine_with_restarts",
    per_device_train_batch_size=4096,
    per_device_eval_batch_size=4096,
    num_train_epochs=600,
    weight_decay=0.001,
    save_strategy="no",
    eval_strategy="steps",
    eval_steps=0.05,
    logging_strategy="steps",
    logging_steps=0.05,
    fp16=False,  # do not need memory savings
    dataloader_num_workers=os.cpu_count(),  # since big batches
    dataloader_pin_memory=True,
    dataloader_persistent_workers=True,
)

# reinstantiate a clean model
model = RatingsModel()
_ = model.to('cuda:0')

trainer = RegressionTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate_fn,
)

trainer.can_return_loss = True

In [10]:
trainer.train()

Step,Training Loss,Validation Loss
1650,2.647900,1.107229
3300,1.206500,1.074908
4950,1.097500,1.080159
6600,1.030300,1.080287
8250,0.988900,1.075464
9900,0.962400,1.081126
11550,0.941200,1.079353
13200,0.925600,1.078243
14850,0.911500,1.080965
16500,0.900200,1.078170


TrainOutput(global_step=33000, training_loss=1.011518858013731, metrics={'train_runtime': 1053.8327, 'train_samples_per_second': 126710.061, 'train_steps_per_second': 31.314, 'total_flos': 0.0, 'train_loss': 1.011518858013731, 'epoch': 600.0})

Write logs as a parquet for later visualization. The steps logged match what's above, although the evals are a separate item and need to be consolidated (annoyingly).

In [11]:
logs = trainer.state.log_history

logs[0:4]

[{'loss': 2.6479,
  'grad_norm': 0.6344577074050903,
  'learning_rate': 0.001987703228645653,
  'epoch': 30.0,
  'step': 1650},
 {'eval_loss': 1.1072291135787964,
  'eval_runtime': 0.2394,
  'eval_samples_per_second': 83536.563,
  'eval_steps_per_second': 20.884,
  'epoch': 30.0,
  'step': 1650},
 {'loss': 1.2065,
  'grad_norm': 0.6202744841575623,
  'learning_rate': 0.0019510859303344694,
  'epoch': 60.0,
  'step': 3300},
 {'eval_loss': 1.0749081373214722,
  'eval_runtime': 0.2262,
  'eval_samples_per_second': 88416.823,
  'eval_steps_per_second': 22.104,
  'epoch': 60.0,
  'step': 3300}]

In [12]:
logs_consolidated = []
i = 0
while i < len(logs)-1:
    base_log = logs[i]
    base_log.update(logs[i+1])
    logs_consolidated.append(base_log)
    i += 2
    
df_logs = pl.DataFrame(logs_consolidated).sort("epoch")
df_logs.write_parquet("mlp_train_logs.parquet")
df_logs

loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second
f64,f64,f64,f64,i64,f64,f64,f64,f64
2.6479,0.634458,0.001988,30.0,1650,1.107229,0.2394,83536.563,20.884
1.2065,0.620274,0.001951,60.0,3300,1.074908,0.2262,88416.823,22.104
1.0975,0.613231,0.001891,90.0,4950,1.080159,0.2245,89090.478,22.273
1.0303,0.621995,0.001809,120.0,6600,1.080287,0.2353,85005.518,21.251
0.9889,0.567169,0.001707,150.0,8250,1.075464,0.2322,86147.804,21.537
…,…,…,…,…,…,…,…,…
0.8547,0.725208,0.000191,480.0,26400,1.076313,0.259,77206.413,19.302
0.8513,0.688252,0.000109,510.0,28050,1.076577,0.2517,79453.301,19.863
0.8485,0.604255,0.000049,540.0,29700,1.07612,0.2603,76838.879,19.21


Save the model weights, which is the artifact we would use to hypothetically deploy the model to production.

In [13]:
from safetensors.torch import save_model

save_model(model, "imdb_embeddings_mlp.safetensors")

## Test Model


In [14]:
_ = model.to("cuda:0").eval()  # to disable BatchNorm1D

eval_dataset = torch.stack([f[0] for f in test_dataset])
actual_values = torch.stack([f[1] for f in test_dataset])

with torch.no_grad():
    output = model(x=eval_dataset.to("cuda:0"))
    preds = output.detach().cpu().numpy()

test_results = (pl.DataFrame({"Predicted": preds, "Actual": actual_values.cpu().numpy()})
                .with_columns(
                    abs_diff=(pl.col("Predicted") - pl.col("Actual")).abs(),
                    square_error = ((pl.col("Actual") - pl.col("Predicted")) ** 2)
                )
               )
                
test_results

Predicted,Actual,abs_diff,square_error
f32,f32,f32,f32
7.114032,7.1,0.014032,0.000197
6.102827,6.5,0.397173,0.157746
3.916766,4.1,0.183233,0.033575
5.002928,5.5,0.497072,0.24708
6.98966,7.2,0.21034,0.044243
…,…,…,…
6.425658,6.2,0.225658,0.050922
6.200366,3.2,3.000366,9.002196
6.106409,5.7,0.406409,0.165168


In [15]:
# Mean Absolute Error (MAE)
test_results["abs_diff"].mean()

0.7786143642246723

In [16]:
# Mean Square Error (MSE)
test_results["square_error"].mean()

1.077159608136191